> ## ⚠️ Important Lab Disclaimer
> This lab requires API keys (OpenAI, Tavily, OpenWeather) if you want to run the live examples.  
> These keys stay only inside the **temporary Udemy lab workspace** and are **not persisted or shared**, but you should still treat them like any other secret.  
> Use your own discretion when entering keys, and feel free to **revoke or rotate** them after the lab.  
> If you prefer **not** to use real API keys, Copy paste away or download the notebook and run it locally.


In [ ]:
# ! pip install langchain-openai langchain-core python-dotenv
# ! pip install langchain


This script securely collects three API keys (`OPENAI_API_KEY`, `TAVILY_API_KEY`, `OPENWEATHER_API_KEY`) using `getpass` so your input is hidden.  
It then creates/overwrites a `.env` file and writes each key as `NAME=value` on its own line.  
Finally, it prints **“Saved .env”** when finished.


In [2]:
from getpass import getpass

names = ["OPENAI_API_KEY", "TAVILY_API_KEY", "OPENWEATHER_API_KEY"]
with open(".env", "w", encoding="utf-8") as f:
    for n in names:
        f.write(f"{n}={getpass(f'Enter {n}: ').strip()}\n")
print("Saved .env")


Saved .env


This code locates the nearest `.env` file in the current working directory (using `find_dotenv`) and loads its key–value pairs into the environment (using `load_dotenv`).  
It does **not** overwrite any existing environment variables because `override=False`.


In [3]:
from dotenv import load_dotenv, find_dotenv
env_path = find_dotenv(usecwd=True)
load_dotenv(env_path, override=False)

True

This code creates an OpenAI chat model (`ChatOpenAI`) with a fixed temperature of 0, sends it a prompt about planning a short New York trip, and returns the model’s response.  
`response.content` gives the raw text, and `response.pretty_print()` displays the response in a cleaner, formatted way.


In [4]:
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

You can swap **any supported chat model** 

LangChain lets you swap chat models easily—just change the import and the model name. Here are concise examples showing different providers:

```python
# OpenAI
from langchain_openai import ChatOpenAI
chat = ChatOpenAI(model="gpt-4o-mini")

# Anthropic (Claude)
from langchain_anthropic import ChatAnthropic
chat = ChatAnthropic(model="claude-3-haiku")

# Google (Gemini)
from langchain_google_genai import ChatGoogleGenerativeAI
chat = ChatGoogleGenerativeAI(model="gemini-1.5-flash")

# Mistral AI
from langchain_mistralai import ChatMistralAI
chat = ChatMistralAI(model="mistral-small-latest")

# Groq (fast inference with open-weights models)
from langchain_groq import ChatGroq
chat = ChatGroq(model="mixtral-8x7b")
```


This sends a prompt to the chat model and returns its generated response.

In [5]:
response = chat.invoke("Plan a simple 1-night/2-day trip to New York for a family of four.")

In [6]:
response

AIMessage(content='Day 1:\n- Arrive in New York City in the morning and check into a family-friendly hotel in Midtown Manhattan.\n- Head to Times Square to take in the bustling atmosphere and grab a quick lunch at one of the many restaurants in the area.\n- Visit the iconic Empire State Building for panoramic views of the city.\n- Walk to Bryant Park and enjoy some leisure time exploring the park and maybe even catching a free outdoor movie or performance.\n- Have dinner at a family-friendly restaurant in the area before heading back to the hotel for the night.\n\nDay 2:\n- Start the day with a visit to Central Park, where you can rent bikes or take a leisurely stroll through the park.\n- Head to the American Museum of Natural History to explore the exhibits and learn about the natural world.\n- Grab lunch at a nearby cafe or food truck before heading to the Museum of Modern Art to see some world-renowned works of art.\n- Spend the afternoon shopping and exploring Fifth Avenue, stoppin

In [7]:
response.content

'Day 1:\n- Arrive in New York City in the morning and check into a family-friendly hotel in Midtown Manhattan.\n- Head to Times Square to take in the bustling atmosphere and grab a quick lunch at one of the many restaurants in the area.\n- Visit the iconic Empire State Building for panoramic views of the city.\n- Walk to Bryant Park and enjoy some leisure time exploring the park and maybe even catching a free outdoor movie or performance.\n- Have dinner at a family-friendly restaurant in the area before heading back to the hotel for the night.\n\nDay 2:\n- Start the day with a visit to Central Park, where you can rent bikes or take a leisurely stroll through the park.\n- Head to the American Museum of Natural History to explore the exhibits and learn about the natural world.\n- Grab lunch at a nearby cafe or food truck before heading to the Museum of Modern Art to see some world-renowned works of art.\n- Spend the afternoon shopping and exploring Fifth Avenue, stopping at iconic stores

In [8]:
response.pretty_print() 

================================== Ai Message ==================================

Day 1:
- Arrive in New York City in the morning and check into a family-friendly hotel in Midtown Manhattan.
- Head to Times Square to take in the bustling atmosphere and grab a quick lunch at one of the many restaurants in the area.
- Visit the iconic Empire State Building for panoramic views of the city.
- Walk to Bryant Park and enjoy some leisure time exploring the park and maybe even catching a free outdoor movie or performance.
- Have dinner at a family-friendly restaurant in the area before heading back to the hotel for the night.

Day 2:
- Start the day with a visit to Central Park, where you can rent bikes or take a leisurely stroll through the park.
- Head to the American Museum of Natural History to explore the exhibits and learn about the natural world.
- Grab lunch at a nearby cafe or food truck before heading to the Museum of Modern Art to see some world-renowned works of art.
- Spend the af

This utility provides a single function, `show_any`, that **pretty-prints any LLM output** in notebooks.  
It detects the object type (messages, LangGraph state dicts, streams, or raw OpenAI responses), formats them with Markdown/JSON, optionally truncates long text, and displays metadata like tool calls and usage when available.


In [9]:
from IPython.display import Markdown, display
from typing import Any, Iterable
import json, inspect

def _md(s: str): display(Markdown(s))
def _j(obj): return "```json\n" + json.dumps(obj, indent=2, default=str) + "\n```"

def show_any(result: Any, show_usage=True, show_meta=True, truncate: int | None = None):
    """
    Pretty-print *anything* you might get back from an LLM call:
    - LangChain Messages / LangGraph state dicts
    - Lists/streams of messages
    - Raw OpenAI SDK responses (chat/responses)
    """
    # 0) Optional simple truncation of big contents
    def _maybe_trunc(s: str) -> str:
        if truncate and isinstance(s, str) and len(s) > truncate:
            return s[:truncate] + f"\n… [truncated {len(s)-truncate} chars]"
        return s

    # A) LangGraph/agent state: {"messages": [...]}
    if isinstance(result, dict) and "messages" in result and isinstance(result["messages"], Iterable):
        _md("### Conversation")
        for i, msg in enumerate(result["messages"], 1):
            role = getattr(msg, "type", msg.__class__.__name__)
            _md(f"**[{i}] {role}**")
            content = getattr(msg, "content", None)
            if content is not None:
                _md(_maybe_trunc(content) if isinstance(content, str) else _j(content))

            # Tool calls / function_call live in different places across providers
            tool_calls = getattr(msg, "tool_calls", None)
            ak = getattr(msg, "additional_kwargs", {}) or {}
            func_call = ak.get("function_call") or ak.get("tool_calls")
            if tool_calls or func_call:
                _md("**Tool calls**")
                _md(_j(tool_calls or func_call))

            # ToolMessage info
            name = getattr(msg, "name", None)
            if name: _md(f"*tool name:* `{name}`")

            if show_meta:
                meta = getattr(msg, "response_metadata", None)
                if meta: _md("**response_metadata**\n" + _j(meta))
        if show_usage and hasattr(result, "usage_metadata") and result.usage_metadata:
            _md("### Usage / metadata\n" + _j(result.usage_metadata))
        return

    # B) Single LangChain message (AIMessage, HumanMessage, ToolMessage, etc.)
    if hasattr(result, "content") or hasattr(result, "response_metadata"):
        role = getattr(result, "type", result.__class__.__name__)
        _md(f"### {role}")
        content = getattr(result, "content", None)
        if content is not None:
            _md(_maybe_trunc(content) if isinstance(content, str) else _j(content))
        tool_calls = getattr(result, "tool_calls", None)
        ak = getattr(result, "additional_kwargs", {}) or {}
        func_call = ak.get("function_call") or ak.get("tool_calls")
        if tool_calls or func_call:
            _md("**Tool calls**\n" + _j(tool_calls or func_call))
        name = getattr(result, "name", None)
        if name: _md(f"*tool name:* `{name}`")
        if show_meta:
            meta = getattr(result, "response_metadata", None)
            if meta: _md("**response_metadata**\n" + _j(meta))
        if show_usage and hasattr(result, "usage_metadata") and result.usage_metadata:
            _md("### Usage / metadata\n" + _j(result.usage_metadata))
        return

    # C) Iterable of messages/chunks (streams, lists)
    if isinstance(result, Iterable) and not isinstance(result, (str, bytes, dict)):
        for item in result: show_any(item, show_usage=show_usage, show_meta=show_meta, truncate=truncate)
        return

    # D) Raw OpenAI SDK objects (chat.completions / responses)
    # Heuristic: has .id and .choices or .output
    if hasattr(result, "id") and (hasattr(result, "choices") or hasattr(result, "output")):
        _md("### OpenAI raw response")
        rid = getattr(result, "id", None)
        model = getattr(result, "model", None)
        if rid or model: _md(_j({"id": rid, "model": model}))
        if hasattr(result, "choices"):  # chat.completions
            for i, ch in enumerate(result.choices, 1):
                _md(f"**Choice {i} — finish_reason:** `{getattr(ch, 'finish_reason', None)}`")
                msg = getattr(ch, "message", None)
                if msg and getattr(msg, "content", None):
                    _md(_maybe_trunc(msg.content))
                if getattr(msg, "tool_calls", None):
                    _md("**Tool calls**\n" + _j(msg.tool_calls))
        elif hasattr(result, "output"):  # responses.create
            _md("**Output**\n" + _j(result.output))
        return

    # E) Fallback: show whatever it is as JSON
    try:
        _md(_j(result if not inspect.isgenerator(result) else list(result)))
    except Exception:
        _md(f"```\n{str(result)}\n```")


In [10]:
show_any(response)

### ai

Day 1:
- Arrive in New York City in the morning and check into a family-friendly hotel in Midtown Manhattan.
- Head to Times Square to take in the bustling atmosphere and grab a quick lunch at one of the many restaurants in the area.
- Visit the iconic Empire State Building for panoramic views of the city.
- Walk to Bryant Park and enjoy some leisure time exploring the park and maybe even catching a free outdoor movie or performance.
- Have dinner at a family-friendly restaurant in the area before heading back to the hotel for the night.

Day 2:
- Start the day with a visit to Central Park, where you can rent bikes or take a leisurely stroll through the park.
- Head to the American Museum of Natural History to explore the exhibits and learn about the natural world.
- Grab lunch at a nearby cafe or food truck before heading to the Museum of Modern Art to see some world-renowned works of art.
- Spend the afternoon shopping and exploring Fifth Avenue, stopping at iconic stores like Tiffany & Co. and Saks Fifth Avenue.
- End the trip with a visit to Rockefeller Center to see the famous ice skating rink and maybe catch a show at Radio City Music Hall before heading back home.

**response_metadata**
```json
{
  "token_usage": {
    "completion_tokens": 247,
    "prompt_tokens": 26,
    "total_tokens": 273,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  },
  "model_name": "gpt-3.5-turbo-0125",
  "system_fingerprint": null,
  "id": "chatcmpl-CgdjgJRRisyMUTpX2j1K2NOQaOzhF",
  "service_tier": "default",
  "finish_reason": "stop",
  "logprobs": null
}
```

### Usage / metadata
```json
{
  "input_tokens": 26,
  "output_tokens": 247,
  "total_tokens": 273,
  "input_token_details": {
    "audio": 0,
    "cache_read": 0
  },
  "output_token_details": {
    "audio": 0,
    "reasoning": 0
  }
}
```

This code prints key metadata from an LLM response: model name, finish reason, prompt/output/total token counts, and the response’s unique run ID. It reads these values from `response.response_metadata` and displays them in a simple, readable format.


In [11]:

print("\n=== KEY META ===")
meta = response.response_metadata 
usage = meta.get("token_usage")
print("Model       :", meta.get("model_name"))
print("Finish      :", meta.get("finish_reason"))  # 'stop' means ended naturally
print("Prompt toks :", usage.get("prompt_tokens"))
print("Output toks :", usage.get("completion_tokens"))
print("Total toks  :", usage.get("total_tokens"))
print("Run ID      :", response.id)


=== KEY META ===
Model       : gpt-3.5-turbo-0125
Finish      : stop
Prompt toks : 26
Output toks : 247
Total toks  : 273
Run ID      : run--660e7b73-67e2-4ca1-a44d-477a0a9a262f-0


This builds a message list with a system role and a human query, sends both to the chat model, and prints the model’s final response content.


In [12]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="You are a concise, friendly travel planner. Prioritize kid-friendly options when asked."),
    HumanMessage(content="Plan a 1N/2D trip to New York for 2 adults + 2 kids.")
]
response = chat.invoke(messages)
print(response.content)


Day 1:
- Morning: Visit Central Park for a leisurely walk, playgrounds, and maybe a boat ride at the lake.
- Afternoon: Head to the American Museum of Natural History for interactive exhibits and the famous dinosaur skeletons.
- Evening: Explore Times Square for its bright lights, street performers, and family-friendly restaurants.

Day 2:
- Morning: Visit the Statue of Liberty and Ellis Island for a dose of history and amazing views of the city skyline.
- Afternoon: Explore the Intrepid Sea, Air & Space Museum for hands-on exhibits of historic aircraft and ships.
- Evening: Enjoy a Broadway show suitable for the whole family.

Remember to check for any age restrictions or special events at each attraction before your visit. Have a great trip!


This creates two chat models with different temperatures (“spicy” and “calm”), sends them the same system and human messages, and prints how each model responds differently based on creativity settings.


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.messages import SystemMessage, HumanMessage

spicy = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.9)
calm  = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0.0)

messages = [
    SystemMessage(content="You are a fun, imaginative assistant."),
    HumanMessage(content="Invent a quirky one-sentence slogan for a coffee shop."),
]

print("Spicy version:\n", spicy.invoke(messages).content)
print("\nCalm version:\n", calm.invoke(messages).content)


Spicy version:
 "Perk up your day with a brew-tiful cup of joy!"

Calm version:
 "Awake your inner wizard with every sip at Brewed Spells Coffee Co."


This streams the model’s response chunk-by-chunk and prints each piece as it arrives, showing how streaming output unfolds in real time.


In [14]:
# Streaming demo
from sys import stdout
for ch in calm.stream(messages):
    print(ch.content or "")
stdout.write("\n")



"
Aw
ake
 your
 inner
 wizard
 with
 every
 sip
 at
 Brew
ed
 Spells
 Coffee
 Co
."





1